In [1]:
import os
from scapy.all import *
import numpy as np
import re
import binascii
import codecs

In [2]:
packets = rdpcap('mqtt_packets_tcpdump.pcap')

In [3]:
# Let's iterate through every packet
tcpPackets = []
for packet in packets:
#    # We're only interested packets with a DNS Round Robin layer
    #print(type(packet))
    if packet.haslayer(TCP):
        #print(packet.load)
        tcpPackets.append(packet)

In [4]:
def charToHex(a):
    hexList = []
    for c in a:
        newHex = str(hex(ord(c)))
        hexList.append(newHex.replace("0x",""))
    return hexList

In [5]:
def fixHex(packet):
    allHex = []
    cleanPacket = str(packet)[2:-1]
    splitPacket = cleanPacket.split("\\")
    for a in splitPacket:
        #print("________________" + a + "_____________")
        reg = re.findall(r"[xX][0-9a-fA-F]{2}",a)
        if reg:
            allHex.append(reg[0].replace("x",""))
            leftOver = a.replace(reg[0],"")
            #c = codecs.encode(b, "hex")
            #print(reg)
            newHex = charToHex(leftOver)
            for n in newHex:
                allHex.append(n.replace("0x",""))
        else:
            newHex = charToHex(a)
            for n in newHex:
                allHex.append(n.replace("0x",""))
        #print(allHex)
    return(allHex)

        #print(reg)

In [6]:
class TcpPacket:
    def __init__(self,packet):           #here packet should be a list of hex values
        self.entirePacket = packet
        self.destination = packet[:6]    #first six bytes
        self.source = packet[6:12]             #next six bytes
        self.coreProtocol = packet[12:14]      #next 2 bytes
        self.ipVersion = packet[14:34]         #next 20 bytes are ipv4 stuff
        self.tcp = packet[34:66]               #next 32 bytes defines TCP protocol
        self.mqtt = packet[66:]

        self.mqttType = self.typeMap(self.mqtt[0])
        print(self.mqttType)
        if self.mqttType[0] == 4 or self.mqttType[0] == 7:
            self.mqttPacket = MqttPublish(self.mqtt)
        else:
            self.mqttPacket = None     #rest of packet will be MQTT
                        
    def typeMap(self,hexVal):
        meanings = {"10":0,"20":1,"82":2,"90":3,"30":4,"c0":5,"d0":6,"31":7}
        hexToCommand = {0:"Connect Command",1:"Connect Ack",2:"Suscribe Request",3:"Suscribe Ack",4:"Publisher Message",5:"Ping Request",6:"Ping Response",7:"Publisher Message (Retain)"}
        a = meanings[hexVal]
        b = hexToCommand[a]
        return(a,b)
    
    def rebuildPacket(self):
        newPacket = self.entirePacket[:66]
        newPacket = newPacket + self.mqttPacket.getHex()
        return " ".join(newPacket)

In [7]:
#class for the mqtt Connect Command
class MqttPublish:
    def __init__(self,mqtt):
        self.entirePacket = mqtt
        code,word = self.typeMap(mqtt[0])
        self.messageType = mqtt[0]
        self.messageTypeWord = word
        self.messageLength = mqtt[1]
        self.messageLengthNum = self.hexToInt(self.messageLength)
        self.topicLength = mqtt[2:4]
        self.topicLengthNum = self.hexToInt(self.topicLength)
        self.topicName = self.findTopicName(self.topicLength,mqtt)
        self.message = mqtt[self.topicLengthNum+4:][:self.messageLengthNum-self.topicLengthNum-3]
        self.messageWords = codecs.decode("".join(self.message),"hex")
        self.messageWords = codecs.decode(self.messageWords,"utf-8")
        if (len(packetList[8][66:])-self.messageLengthNum-2) == 0:
            self.disconnect=False
        else:
            self.disconnect=True
        
    def findTopicName(self,topicLength,mqtt):
        length = self.hexToInt(topicLength) #convert hex string to int
        topicName = mqtt[4:length+4]
        return(topicName)
    
    def hexToInt(self,length):
        return(int("".join(length),16))
    
    def typeMap(self,hexVal):
        meanings = {"10":0,"20":1,"82":2,"90":3,"30":4,"c0":5,"d0":6,"31":7}
        hexToCommand = {0:"Connect Command",1:"Connect Ack",2:"Suscribe Request",3:"Suscribe Ack",4:"Publisher Message",5:"Ping Request",6:"Ping Response",7:"Publisher Message (Retain)"}
        a = meanings[hexVal]
        b = hexToCommand[a]
        return(a,b)
    
    def changeMessage(self,newMessage): #since MQTT doesn't have a checksum we can just go ahead and alter the packet
        self.messageWords = newMessage
        self.messageLengthNum = self.messageLengthNum - (self.messageLengthNum-len(newMessage))
        self.messageLength = hex(self.messageLengthNum)
        message = []
        for letter in newMessage:
            message.append(hex(ord(letter)))
        self.message = message
        
    def getHex(self):
        full = []
        meanings = {0:"10",1:"20",2:"82",3:"90",4:"30",5:"c0",6:"d0",7:"31"}
        [full.append(meanings[int(x)]) for x in self.messageType]
        a = self.messageLength.replace("0x","")
        if len(a) < 2:
            a = '0' + a
        full.append(a)
        #[full.append(x) for x in a]
        [full.append(x) for x in self.topicLength]
        [full.append(x) for x in self.topicName]
        [full.append(x.replace("0x","")) for x in self.message]
        if self.disconnect == True:
            full.append("e0")
            full.append("00")
        return(full)
        
        
        

In [8]:
def onlyMQTTPackets(packets):
    processedPackets = []
    for packet in packetList:
        processedPackets.append(TcpPacket(packet))
        
    messageRequests = []
    for pPacket in processedPackets:
        num,word = pPacket.mqttType
        if num == 4 | num == 7:
            messageRequests.append(pPacket)
    return(messageRequests)

In [9]:
packetList = []
for packet in tcpPackets:
    packetList.append(fixHex(packet))

In [10]:
msgPackets = onlyMQTTPackets(packetList)

(0, 'Connect Command')
(1, 'Connect Ack')
(2, 'Suscribe Request')
(3, 'Suscribe Ack')
(7, 'Publisher Message (Retain)')
(5, 'Ping Request')
(6, 'Ping Response')
(0, 'Connect Command')
(4, 'Publisher Message')
(1, 'Connect Ack')
(4, 'Publisher Message')
(5, 'Ping Request')
(6, 'Ping Response')
(5, 'Ping Request')
(6, 'Ping Response')
(5, 'Ping Request')
(6, 'Ping Response')
(5, 'Ping Request')
(6, 'Ping Response')


In [11]:
a = msgPackets[0]

In [14]:
a.mqttPacket.messageWords

'28'

In [15]:
a.mqttPacket.changeMessage("80")

In [16]:
a.mqttPacket.messageWords

'80'

In [17]:
a.rebuildPacket()

'28 cf e9 21 14 8f 24 a2 e1 e6 ee 9b 08 00 45 20 00 66 a7 a0 00 00 29 06 f9 b3 c6 29 1e f1 6e 00 01 04 07 5b c0 af 79 3c 11 ec c1 e1 ff 69 80 18 00 e3 82 ac 00 00 01 01 08 6e 38 ac c3 56 38 21 d3 61 90 20 02 00 0b 53 61 6d 70 6c 65 54 6f 70 69 63 38 30 e0 00'